In [127]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [128]:
# Load the training and testing sets from a pickle file
with open('../data/datasets.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [129]:
# Convert y_train and y_test to numpy arrays
y_train = np.array(y_train, copy=True)
y_test = np.array(y_test, copy=True)